In [130]:
import csv
import pandas as pd
from pandas import *
from __future__ import division
from pandas.tseries.offsets import *
import dateutil.rrule as dr
import dateutil.parser as dp
import dateutil.relativedelta as drel
import numpy as np

## Loading files

In [131]:
#spd = read_csv('http://real-chart.finance.yahoo.com/table.csv?s=%5EGSPC&a=00&b=1&c=1974&d=05&e=9&f=2015&g=d&ignore=.csv',
#               delimiter=',', header=0, 
#               names=['date','o','h','l','close','vol','c'])
#spw = read_csv('http://real-chart.finance.yahoo.com/table.csv?s=%5EGSPC&a=00&b=1&c=1974&d=05&e=9&f=2015&g=w&ignore=.csv',
#               delimiter=',', header=0, 
#               names=['date','o','h','l','close','vol','c'])

#http://real-chart.finance.yahoo.com/table.csv?s=DAL&a=04&b=3&c=2008&d=05&e=9&f=2015&g=d&ignore=.csv
#http://real-chart.finance.yahoo.com/table.csv?s=%5EGSPC&a=00&b=1&c=1974&d=05&e=9&f=2015&g=d&ignore=.csv

In [132]:
spd = read_csv('spd.csv', delimiter=',', header=0)
spw = read_csv('spw.csv', delimiter=',', header=0)
options = read_csv('options.csv', delimiter=',', header=0)

In [133]:
spd.rename(columns={'Unnamed: 0':'nrow'}, inplace=True)
spw.rename(columns={'Unnamed: 0':'nrow'}, inplace=True)
options.rename(columns={'Unnamed: 0':'nrow'}, inplace=True)

In [134]:
spd['date'] = pd.to_datetime(spd['date'])
spw['date'] = pd.to_datetime(spw['date'])
print type(spd.date[0]), type(spw.date[0])

<class 'pandas.tslib.Timestamp'> <class 'pandas.tslib.Timestamp'>


In [135]:
spd.head(3)

,nrow,date,o,h,l,close,vol,c
0,0,2015-06-05,2095.09009,2100.98999,2085.66992,2092.83008,3243690000,2092.83008
1,1,2015-06-04,2112.35010,2112.88989,2093.22998,2095.84009,3200050000,2095.84009
2,2,2015-06-03,2110.63989,2121.91992,2109.61011,2114.07007,3099980000,2114.07007


In [136]:
spd.tail(3)

,nrow,date,o,h,l,close,vol,c
10449,10449,1974-01-04,99.80,100.70,97.70,98.90,21700000,98.90
10450,10450,1974-01-03,98.02,100.94,98.02,99.80,24850000,99.80
10451,10451,1974-01-02,97.55,98.38,96.25,97.68,12060000,97.68


In [137]:
spw.head(3)

,nrow,date,o,h,l,close,vol,c
0,0,2015-06-01,2108.63989,2121.91992,2085.66992,2092.83008,3120956000,2092.83008
1,1,2015-05-26,2125.34009,2126.21997,2099.17993,2107.38989,3344457500,2107.38989
2,2,2015-05-18,2121.30005,2134.71997,2120.01001,2126.06006,2970484000,2126.06006


In [138]:
spw.tail(3)

,nrow,date,o,h,l,close,vol,c
2159,2159,1974-01-14,93.66,98.35,92.35,95.56,16060000,95.56
2160,2160,1974-01-07,98.90,99.31,91.62,93.66,17296000,93.66
2161,2161,1974-01-02,97.55,100.94,96.25,98.90,19536600,98.90


In [139]:
len(spd)

10452

In [140]:
len(spw)

2162

In [248]:
spd.date[spd.date == dp.parse("20/03/2008")] = pd.to_datetime(dp.parse("21/03/2008"))

/usr/local/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [141]:
for i in range(len(spw.date)):
    spw.date[i] = spw.date[i] + DateOffset(weekday = 4) + DateOffset(days = 1) - BDay()
#    if spw.date[i] == dp.parse("21/03/2008"):
#        spw.date[i] = pd.to_datetime(dp.parse("20/03/2008"))

/usr/local/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app


In [142]:
spw.head(3)

,nrow,date,o,h,l,close,vol,c
0,0,2015-06-05,2108.63989,2121.91992,2085.66992,2092.83008,3120956000,2092.83008
1,1,2015-05-29,2125.34009,2126.21997,2099.17993,2107.38989,3344457500,2107.38989
2,2,2015-05-22,2121.30005,2134.71997,2120.01001,2126.06006,2970484000,2126.06006


In [143]:
spw.tail(3)

,nrow,date,o,h,l,close,vol,c
2159,2159,1974-01-18,93.66,98.35,92.35,95.56,16060000,95.56
2160,2160,1974-01-11,98.90,99.31,91.62,93.66,17296000,93.66
2161,2161,1974-01-04,97.55,100.94,96.25,98.90,19536600,98.90


## Moving Averages

In [144]:
d12 = []
d24 = []
d48 = []
d72 = []
d96 = []
d192 = []
d288 = []
for i in range(len(spd)-288):
    d12.append(sum(spd.c[i:(i+12)])/12)
    d24.append(sum(spd.c[i:(i+24)])/24)
    d48.append(sum(spd.c[i:(i+48)])/48)
    d72.append(sum(spd.c[i:(i+72)])/72)
    d96.append(sum(spd.c[i:(i+96)])/96)
    d192.append(sum(spd.c[i:(i+192)])/192)
    d288.append(sum(spd.c[i:(i+288)])/288)
    
d192[-1]

75.564843749999994

In [145]:
w12 = []
w24 = []
w48 = []
w72 = []
w96 = []
w192 = []
w288 = []
for i in range(len(spw)-288):
    w12.append(sum(spw.c[i:(i+12)])/12)
    w24.append(sum(spw.c[i:(i+24)])/24)
    w48.append(sum(spw.c[i:(i+48)])/48)
    w72.append(sum(spw.c[i:(i+72)])/72)
    w96.append(sum(spw.c[i:(i+96)])/96)
    w192.append(sum(spw.c[i:(i+192)])/192)
    w288.append(sum(spw.c[i:(i+288)])/288)
    
w192[-1]

98.690937500000004

## Indicators

In [146]:
ltd = []
for i in range(len(spd)-288):
    t12 = ((spd.c[i]/(sum(spd.c[(i+6):(i+12)])/6)-1)*100)
    t24 = ((spd.c[i]/(sum(spd.c[(i+18):(i+24)])/6)-1)*100)
    t48 = ((spd.c[i]/(sum(spd.c[(i+42):(i+48)])/6)-1)*100)
    t72 = ((spd.c[i]/(sum(spd.c[(i+66):(i+72)])/6)-1)*100)
    t96 = ((spd.c[i]/(sum(spd.c[(i+90):(i+96)])/6)-1)*100)
    t192 = ((spd.c[i]/(sum(spd.c[(i+186):(i+192)])/6)-1)*100)
    t288 = ((spd.c[i]/(sum(spd.c[(i+282):(i+288)])/6)-1)*100)
    ltd.append((t12+t24+t48+t72+t96+t192+t288)/7)
ltd.extend([0]*288)

In [147]:
ltw = []
for i in range(len(spw)-288):
    t12 = ((spw.c[i]/(sum(spw.c[(i+6):(i+12)])/6)-1)*100)
    t24 = ((spw.c[i]/(sum(spw.c[(i+18):(i+24)])/6)-1)*100)
    t48 = ((spw.c[i]/(sum(spw.c[(i+42):(i+48)])/6)-1)*100)
    t72 = ((spw.c[i]/(sum(spw.c[(i+66):(i+72)])/6)-1)*100)
    t96 = ((spw.c[i]/(sum(spw.c[(i+90):(i+96)])/6)-1)*100)
    t192 = ((spw.c[i]/(sum(spw.c[(i+186):(i+192)])/6)-1)*100)
    t288 = ((spw.c[i]/(sum(spw.c[(i+282):(i+288)])/6)-1)*100)
    ltw.append((t12+t24+t48+t72+t96+t192+t288)/7)
ltw.extend([0]*288)

In [148]:
print max(ltd), min(ltd), max(ltw), min (ltw)

23.3328730247 -35.6366113852 71.7672905577 -41.5810396385


In [149]:
print ltd.index(max(ltd)), ltd.index(min(ltd)), ltw.index(max(ltw)), ltw.index(min(ltw))

8089 1644 881 326


In [150]:
lc288d = []
lc72d = []
for i in range(len(spd)-288):
    min12 = ((spd.c[i]/min(spd.c[(i+6):(i+12)]))-1)*100
    max12 = ((spd.c[i]/max(spd.c[(i+6):(i+12)]))-1)*100
    min24 = ((spd.c[i]/min(spd.c[(i+18):(i+24)]))-1)*100
    max24 = ((spd.c[i]/max(spd.c[(i+18):(i+24)]))-1)*100
    min48 = ((spd.c[i]/min(spd.c[(i+42):(i+48)]))-1)*100
    max48 = ((spd.c[i]/max(spd.c[(i+42):(i+48)]))-1)*100
    min72 = ((spd.c[i]/min(spd.c[(i+66):(i+72)]))-1)*100
    max72 = ((spd.c[i]/max(spd.c[(i+66):(i+72)]))-1)*100
    min96 = ((spd.c[i]/min(spd.c[(i+90):(i+96)]))-1)*100
    max96 = ((spd.c[i]/max(spd.c[(i+90):(i+96)]))-1)*100
    min192 = ((spd.c[i]/min(spd.c[(i+186):(i+192)]))-1)*100
    max192 = ((spd.c[i]/max(spd.c[(i+186):(i+192)]))-1)*100
    min288 = ((spd.c[i]/min(spd.c[(i+282):(i+288)]))-1)*100
    max288 = ((spd.c[i]/max(spd.c[(i+282):(i+288)]))-1)*100
    
    clong288 = 0
    clong72 = 0
    cshort288 = 0
    cshort72 = 0
    
    contatl = 0
    if spd.c[i] < sum(spd.c[i:i+12])/12:
        contatl += 1
    if spd.c[i] > spd.c[i+12]:
        contatl += 1
    if max12 > 0:
        contatl += 1
    contats = 0
    if spd.c[i] > sum(spd.c[i:i+12])/12:
        contats -= 1
    if spd.c[i] < spd.c[i+12]:
        contats -= 1
    if min12 < 0:
        contats -= 1
    if contatl == 3 or contats == -3:
        c12 = (((sum(spd.c[i:(i+12)])/12)/spd.c[i])-1)*100
    else:
        c12 = 0
    if contatl == 3:
        clong288 += 1
        clong72 += 1
    if contats == -3:
        cshort288 += 1
        cshort72 += 1
        
    contatl = 0
    if spd.c[i] < sum(spd.c[i:i+24])/24:
        contatl +=1
    if spd.c[i] > spd.c[i+24]:
        contatl +=1
    if max24 > 0:
        contatl +=1
    contats = 0
    if spd.c[i] > sum(spd.c[i:i+24])/24:
        contats -=1
    if spd.c[i] < spd.c[i+24]:
        contats -=1
    if min24 < 0:
        contats -=1
    if contatl == 3 or contats == -3:
        c24 = (((sum(spd.c[i:(i+24)])/24)/spd.c[i])-1)*100
    else:
        c24 = 0
    if contatl == 3:
        clong288 +=1
        clong72 +=1
    if contats == -3:
        cshort288 +=1
        cshort72 +=1
        
    contatl = 0
    if spd.c[i] < sum(spd.c[i:i+48])/48:
        contatl +=1
    if spd.c[i] > spd.c[i+48]:
        contatl +=1
    if max48 > 0:
        contatl +=1
    contats = 0
    if spd.c[i] > sum(spd.c[i:i+48])/48:
        contats -=1
    if spd.c[i] < spd.c[i+48]:
        contats -=1
    if min48 < 0:
        contats -=1
    if contatl == 3 or contats == -3:
        c48 = (((sum(spd.c[i:(i+48)])/48)/spd.c[i])-1)*100
    else:
        c48 = 0    
    if contatl == 3:
        clong288 +=1
        clong72 +=1
    if contats == -3:
        cshort288 +=1
        cshort72 +=1
    
    contatl = 0
    if spd.c[i] < sum(spd.c[i:i+72])/72:
        contatl +=1
    if spd.c[i] > spd.c[i+72]:
        contatl +=1
    if max72 > 0:
        contatl +=1
    contats = 0
    if spd.c[i] > sum(spd.c[i:i+72])/72:
        contats -=1
    if spd.c[i] < spd.c[i+72]:
        contats -=1
    if min72 < 0:
        contats -=1
    if contatl == 3 or contats == -3:
        c72 = (((sum(spd.c[i:(i+72)])/72)/spd.c[i])-1)*100
    else:
        c72 = 0  
    if contatl == 3:
        clong288 +=1
        clong72 +=1
    if contats == -3:
        cshort288 +=1
        cshort72 +=1
    
    contatl = 0
    if spd.c[i] < sum(spd.c[i:i+96])/96:
        contatl +=1
    if spd.c[i] > spd.c[i+96]:
        contatl +=1
    if max96 > 0:
        contatl +=1
    contats = 0
    if spd.c[i] > sum(spd.c[i:i+96])/96:
        contats -=1
    if spd.c[i] < spd.c[i+96]:
        contats -=1
    if min96 < 0:
        contats -=1
    if contatl == 3 or contats == -3:
        c96 = (((sum(spd.c[i:(i+96)])/96)/spd.c[i])-1)*100
    else:
        c96 = 0
    if contatl == 3:
        clong288 +=1
    if contats == -3:
        cshort288 +=1
        
    contatl = 0
    if spd.c[i] < sum(spd.c[i:i+192])/192:
        contatl +=1
    if spd.c[i] > spd.c[i+192]:
        contatl +=1
    if max192 > 0:
        contatl +=1
    contats = 0
    if spd.c[i] > sum(spd.c[i:i+192])/192:
        contats -=1
    if spd.c[i] < spd.c[i+192]:
        contats -=1
    if min192 < 0:
        contats -=1
    if contatl == 3 or contats == -3:
        c192 = (((sum(spd.c[i:(i+192)])/192)/spd.c[i])-1)*100
    else:
        c192 = 0      
    if contatl == 3:
        clong288 +=1
    if contats == -3:
        cshort288 +=1
        
    contatl = 0
    if spd.c[i] < sum(spd.c[i:i+288])/288:
        contatl +=1
    if spd.c[i] > spd.c[i+288]:
        contatl +=1
    if max288 > 0:
        contatl +=1
    contats = 0
    if spd.c[i] > sum(spd.c[i:i+288])/288:
        contats -=1
    if spd.c[i] < spd.c[i+288]:
        contats -=1
    if min288 < 0:
        contats -=1
    if contatl == 3 or contats == -3:
        c288 = (((sum(spd.c[i:(i+288)])/288)/spd.c[i])-1)*100
    else:
        c288 = 0
    if contatl == 3:
        clong288 +=1
    if contats == -3:
        cshort288 += 1

    if (clong288 + cshort288) > 0:
        ctot288 = (clong288 + cshort288)
    else:
        ctot288 = 1
    lc288d.append((c12+c24+c48+c72+c96+c192+c288)/ctot288)
    
    if (clong72 + cshort72) > 0:
        ctot72 = (clong72 + cshort72)
    else:
        ctot72 = 1
    lc72d.append((c12+c24+c48+c72)/ctot72)
lc288d.extend([0]*288)
lc72d.extend([0]*288)

In [151]:
print max(lc288d), min(lc288d), max(lc72d), min(lc72d)

17.8730822728 -15.697302014 4.46218397049 -6.65612346649


In [152]:
print lc288d.index(max(lc288d)), lc288d.index(min(lc288d)), lc72d.index(max(lc72d)), lc72d.index(min(lc72d))

6948 1399 9236 1544


In [153]:
print spd.date[1644]

2008-11-20 00:00:00


In [154]:
print lc288d[0], lc72d[0]

0.45128917609 0.45128917609


In [155]:
lc288w = []
lc72w = []
for i in range(len(spw)-288):
    min12 = ((spw.c[i]/min(spw.c[(i+6):(i+12)]))-1)*100
    max12 = ((spw.c[i]/max(spw.c[(i+6):(i+12)]))-1)*100
    min24 = ((spw.c[i]/min(spw.c[(i+18):(i+24)]))-1)*100
    max24 = ((spw.c[i]/max(spw.c[(i+18):(i+24)]))-1)*100
    min48 = ((spw.c[i]/min(spw.c[(i+42):(i+48)]))-1)*100
    max48 = ((spw.c[i]/max(spw.c[(i+42):(i+48)]))-1)*100
    min72 = ((spw.c[i]/min(spw.c[(i+66):(i+72)]))-1)*100
    max72 = ((spw.c[i]/max(spw.c[(i+66):(i+72)]))-1)*100
    min96 = ((spw.c[i]/min(spw.c[(i+90):(i+96)]))-1)*100
    max96 = ((spw.c[i]/max(spw.c[(i+90):(i+96)]))-1)*100
    min192 = ((spw.c[i]/min(spw.c[(i+186):(i+192)]))-1)*100
    max192 = ((spw.c[i]/max(spw.c[(i+186):(i+192)]))-1)*100
    min288 = ((spw.c[i]/min(spw.c[(i+282):(i+288)]))-1)*100
    max288 = ((spw.c[i]/max(spw.c[(i+282):(i+288)]))-1)*100
    
    clong288 = 0
    clong72 = 0
    cshort288 = 0
    cshort72 = 0
    
    contatl = 0
    if spw.c[i] < sum(spw.c[i:i+12])/12:
        contatl += 1
    if spw.c[i] > spw.c[i+12]:
        contatl += 1
    if max12 > 0:
        contatl += 1
    contats = 0
    if spw.c[i] > sum(spw.c[i:i+12])/12:
        contats -= 1
    if spw.c[i] < spw.c[i+12]:
        contats -= 1
    if min12 < 0:
        contats -= 1
    if contatl == 3 or contats == -3:
        c12 = (((sum(spw.c[i:(i+12)])/12)/spw.c[i])-1)*100
    else:
        c12 = 0
    if contatl == 3:
        clong288 += 1
        clong72 += 1
    if contats == -3:
        cshort288 += 1
        cshort72 += 1
        
    contatl = 0
    if spw.c[i] < sum(spw.c[i:i+24])/24:
        contatl +=1
    if spw.c[i] > spw.c[i+24]:
        contatl +=1
    if max24 > 0:
        contatl +=1
    contats = 0
    if spw.c[i] > sum(spw.c[i:i+24])/24:
        contats -=1
    if spw.c[i] < spw.c[i+24]:
        contats -=1
    if min24 < 0:
        contats -=1
    if contatl == 3 or contats == -3:
        c24 = (((sum(spw.c[i:(i+24)])/24)/spw.c[i])-1)*100
    else:
        c24 = 0
    if contatl == 3:
        clong288 +=1
        clong72 +=1
    if contats == -3:
        cshort288 +=1
        cshort72 +=1
        
    contatl = 0
    if spw.c[i] < sum(spw.c[i:i+48])/48:
        contatl +=1
    if spw.c[i] > spw.c[i+48]:
        contatl +=1
    if max48 > 0:
        contatl +=1
    contats = 0
    if spw.c[i] > sum(spw.c[i:i+48])/48:
        contats -=1
    if spw.c[i] < spw.c[i+48]:
        contats -=1
    if min48 < 0:
        contats -=1
    if contatl == 3 or contats == -3:
        c48 = (((sum(spw.c[i:(i+48)])/48)/spw.c[i])-1)*100
    else:
        c48 = 0    
    if contatl == 3:
        clong288 +=1
        clong72 +=1
    if contats == -3:
        cshort288 +=1
        cshort72 +=1
    
    contatl = 0
    if spw.c[i] < sum(spw.c[i:i+72])/72:
        contatl +=1
    if spw.c[i] > spw.c[i+72]:
        contatl +=1
    if max72 > 0:
        contatl +=1
    contats = 0
    if spw.c[i] > sum(spw.c[i:i+72])/72:
        contats -=1
    if spw.c[i] < spw.c[i+72]:
        contats -=1
    if min72 < 0:
        contats -=1
    if contatl == 3 or contats == -3:
        c72 = (((sum(spw.c[i:(i+72)])/72)/spw.c[i])-1)*100
    else:
        c72 = 0  
    if contatl == 3:
        clong288 +=1
        clong72 +=1
    if contats == -3:
        cshort288 +=1
        cshort72 +=1
    
    contatl = 0
    if spw.c[i] < sum(spw.c[i:i+96])/96:
        contatl +=1
    if spw.c[i] > spw.c[i+96]:
        contatl +=1
    if max96 > 0:
        contatl +=1
    contats = 0
    if spw.c[i] > sum(spw.c[i:i+96])/96:
        contats -=1
    if spw.c[i] < spw.c[i+96]:
        contats -=1
    if min96 < 0:
        contats -=1
    if contatl == 3 or contats == -3:
        c96 = (((sum(spw.c[i:(i+96)])/96)/spw.c[i])-1)*100
    else:
        c96 = 0
    if contatl == 3:
        clong288 +=1
    if contats == -3:
        cshort288 +=1
        
    contatl = 0
    if spw.c[i] < sum(spw.c[i:i+192])/192:
        contatl +=1
    if spw.c[i] > spw.c[i+192]:
        contatl +=1
    if max192 > 0:
        contatl +=1
    contats = 0
    if spw.c[i] > sum(spw.c[i:i+192])/192:
        contats -=1
    if spw.c[i] < spw.c[i+192]:
        contats -=1
    if min192 < 0:
        contats -=1
    if contatl == 3 or contats == -3:
        c192 = (((sum(spw.c[i:(i+192)])/192)/spw.c[i])-1)*100
    else:
        c192 = 0      
    if contatl == 3:
        clong288 +=1
    if contats == -3:
        cshort288 +=1
        
    contatl = 0
    if spw.c[i] < sum(spw.c[i:i+288])/288:
        contatl +=1
    if spw.c[i] > spw.c[i+288]:
        contatl +=1
    if max288 > 0:
        contatl +=1
    contats = 0
    if spw.c[i] > sum(spw.c[i:i+288])/288:
        contats -=1
    if spw.c[i] < spw.c[i+288]:
        contats -=1
    if min288 < 0:
        contats -=1
    if contatl == 3 or contats == -3:
        c288 = (((sum(spw.c[i:(i+288)])/288)/spw.c[i])-1)*100
    else:
        c288 = 0
    if contatl == 3:
        clong288 +=1
    if contats == -3:
        cshort288 += 1

    if (clong288 + cshort288) > 0:
        ctot288 = (clong288 + cshort288)
    else:
        ctot288 = 1
    lc288w.append((c12+c24+c48+c72+c96+c192+c288)/ctot288)
    
    if (clong72 + cshort72) > 0:
        ctot72 = (clong72 + cshort72)
    else:
        ctot72 = 1
    lc72w.append((c12+c24+c48+c72)/ctot72)
lc288w.extend([0]*288)
lc72w.extend([0]*288)

In [156]:
print max(lc288w), min(lc288w), max(lc72w), min(lc72w)

42.2414297463 -15.094582949 9.81403250041 -13.8656712844


In [157]:
print lc288w.index(max(lc288w)), lc288w.index(min(lc288w)), lc72w.index(max(lc72w)), lc72w.index(min(lc72w))

662 214 1426 284


In [158]:
print spw.date[214]

2011-04-29 00:00:00


In [159]:
print lc288w[0], lc72w[0]

0.0 0.0


In [160]:
ldd = []
for i in range(len(spd)-288):
    ld = (((d12[i]*12)+(d24[i]*24)+(d48[i]*48)+(d72[i]*72)+(d96[i]*96)+(d192[i]*192)+(d288[i]*288))/732)
    ldd.append(((ld/spd.c[i])-1)*100)
ldd.extend([0]*288)

In [161]:
print max(ldd), min(ldd)

60.3588420227 -16.6628320733


In [162]:
print ldd.index(max(ldd)), ldd.index(min(ldd))

1644 8217


In [163]:
print spd.date[8217]

1982-11-03 00:00:00


In [164]:
print ldd[0]

-2.07431921315


In [165]:
ldw = []
for i in range(len(spw)-288):
    ld = (((w12[i]*12)+(w24[i]*24)+(w48[i]*48)+(w72[i]*72)+(w96[i]*96)+(w192[i]*192)+(w288[i]*288))/732)
    ldw.append(((ld/spw.c[i])-1)*100)
ldw.extend([0]*288)

In [166]:
print max(ldw), min(ldw)

79.73482651 -34.846752505


In [167]:
print ldw.index(max(ldw)), ldw.index(min(ldw))

326 896


In [168]:
print spw.date[326]

2009-03-06 00:00:00


In [169]:
print ldw[0]

-17.8479186214


In [170]:
lmd = []
for i in range(len(spd)-288):
    lm = 0
    if spd.c[i] > d96[i] and spd.c[i] > d288[i]:
        lm = 1
    elif spd.c[i] < d96[i] and spd.c[i] < d288[i]:
        lm = -1
    lmd.append(lm)
lmd.extend([0]*288)

In [171]:
print max(lmd), min(lmd)

1 -1


In [172]:
print sum(lmd)/len(lmd)

0.420780711825


In [173]:
print lmd[0]

1


In [174]:
lmw = []
for i in range(len(spw)-288):
    lm = 0
    if spw.c[i] > w96[i] and spw.c[i] > w288[i]:
        lm = 1
    elif spw.c[i] < w96[i] and spw.c[i] < w288[i]:
        lm = -1
    lmw.append(lm)    
lmw.extend([0]*288)

In [175]:
print max(lmw), min(lmw)

1 -1


In [176]:
print sum(lmw)/len(lmw)

0.551341350601


In [177]:
print lmw[0]

1


In [178]:
spd['lixt'] = ltd
spd['lixc72'] = lc72d
spd['lixc288'] = lc288d
spd['lixd'] = ldd
spd['lixm'] = lmd
spd.head(3)

,nrow,date,o,h,l,close,vol,c,lixt,lixc72,lixc288,lixd,lixm
0,0,2015-06-05,2095.09009,2100.98999,2085.66992,2092.83008,3243690000,2092.83008,2.523671,0.451289,0.451289,-2.074319,1
1,1,2015-06-04,2112.35010,2112.88989,2093.22998,2095.84009,3200050000,2095.84009,2.733611,0.275403,0.275403,-2.240354,1
2,2,2015-06-03,2110.63989,2121.91992,2109.61011,2114.07007,3099980000,2114.07007,3.870495,0.000000,0.000000,-3.114668,1


In [179]:
spw['lixt'] = ltw
spw['lixc72'] = lc72w
spw['lixc288'] = lc288w
spw['lixd'] = ldw
spw['lixm'] = lmw
spw.head(3)

,nrow,date,o,h,l,close,vol,c,lixt,lixc72,lixc288,lixd,lixm
0,0,2015-06-05,2108.63989,2121.91992,2085.66992,2092.83008,3120956000,2092.83008,29.583573,0,0,-17.847919,1
1,1,2015-05-29,2125.34009,2126.21997,2099.17993,2107.38989,3344457500,2107.38989,31.080824,0,0,-18.595005,1
2,2,2015-05-22,2121.30005,2134.71997,2120.01001,2126.06006,2970484000,2126.06006,32.704591,0,0,-19.499769,1


## Building samples dataset

In [180]:
exp = []
d = dp.parse("21/03/2015")
while d > dp.parse("20/03/1980"):
    d = d - WeekOfMonth(weekday = 4, week = 2) + DateOffset(days = 1) - BDay()
    if d != dp.parse("21/09/2001"):
        exp.append(d)
    d = d - DateOffset(days = 80)

len(exp)

140

In [255]:
q1c = []
q1cstd = []
q1high = []
q1low = []
q0c = []
q0cstd = []
q0high = []
q0low = []
q0spanmean = []
q0spanstd = []
q0lixt = []
q0lixc72 = []
q0lixc288 = []
q0lixd = []
q0lixm = []
q0lixtv = []
q0lixc72v = []
q0lixc288v = []
q0lixdv = []
q0lixmv = []
m0c = []
m0cstd = []
m0high = []
m0low = []
m0spanmean = []
m0spanstd = []
m0lixt = []
m0lixc72 = []
m0lixc288 = []
m0lixd = []
m0lixm = []
m0lixtv = []
m0lixc72v = []
m0lixc288v = []
m0lixdv = []
m0lixmv = []

In [256]:
for q1end in exp:
    q1start = q1end - DateOffset(weeks = 13)
    m0start = q1start - DateOffset(weeks = 4)
    q0start = q1start - DateOffset(weeks = 13)
    q1indend = spw.nrow[spw.date == q1end]
    
    q1cstart = float(spw.c[q1indend + 13])
    q1cend = float(spw.c[q1indend])    
    q1cv = ((q1cend/q1cstart)-1)*100
    q1c.append(q1cv)   
    
    q1clist = spd.c[(spd.date >= q1start) & (spd.date<= q1end)]
    q1clist = q1clist.reset_index(drop=True)
    q1cvars = []
    for i in range(len(q1clist)):
        if i < len(q1clist)-1:
            q1cvars.append(((q1clist[i]/q1clist[i+1])-1)*100)
    q1cstd.append(np.std(q1cvars))
    
    q1chigh = ((np.max(q1clist)/q1cstart)-1)*100
    q1high.append(q1chigh)
    
    q1clow = ((np.min(q1clist)/q1cstart)-1)*100
    q1low.append(q1clow)
    
    q1indstart = spw.nrow[spw.date == q1start]
    
    q0cstart = float(spw.c[q1indstart + 13])
    q0cend = float(spw.c[q1indstart])    
    q0cv = ((q0cend/q0cstart)-1)*100
    q0c.append(q0cv)
    
    q0clist = spd.c[(spd.date >= q0start) & (spd.date<= q1start)]
    q0clist = q0clist.reset_index(drop=True)
    q0cvars = []
    for i in range(len(q0clist)):
        if i < len(q0clist)-1:
            q0cvars.append(((q0clist[i]/q0clist[i+1])-1)*100)
    q0cstd.append(np.std(q0cvars))
    
    q0hlist = spd.h[(spd.date >= q0start) & (spd.date<= q1start)]
    q0hlist = q0hlist.reset_index(drop=True)
    q0hvars = []
    for i in range(len(q0hlist)):
        q0hvars.append(((q0hlist[i]/q0clist[i])-1)*100)
    q0high.append(np.mean(q0hvars))
    
    q0llist = spd.l[(spd.date >= q0start) & (spd.date<= q1start)]
    q0llist = q0llist.reset_index(drop=True)
    q0lvars = []
    for i in range(len(q0llist)):
        q0lvars.append(((q0llist[i]/q0clist[i])-1)*100)
    q0low.append(np.mean(q0lvars))
    
    q0svars = []
    for i in range(len(q0llist)):
        q0svars.append(((q0hlist[i]/q0llist[i])-1)*100)
    q0spanmean.append(np.mean(q0svars))
    q0spanstd.append(np.std(q0svars))

    lixt1 = float((spw.lixt[spw.date == q1start]))
    lixt0 = float((spw.lixt[spw.date == q0start]))
    lixtm1 = float((spd.lixt[spd.date == q1start]))
    lixtm0 = float((spd.lixt[spd.date == m0start]))
    q0lixt.append(float(lixt1))
    q0lixtv.append(float(lixt1 - lixt0))
    m0lixt.append(float(lixtm1))
    m0lixtv.append(float(lixtm1 - lixtm0))
    
    lixc721 = float((spw.lixc72[spw.date == q1start]))
    lixc720 = float((spw.lixc72[spw.date == q0start]))
    lixc72m1 = float((spd.lixc72[spd.date == q1start]))
    lixc72m0 = float((spd.lixc72[spd.date == m0start]))
    q0lixc72.append(float(lixc721))
    q0lixc72v.append(float(lixc721 - lixc720))
    m0lixc72.append(float(lixc72m1))
    m0lixc72v.append(float(lixc72m1 - lixc72m0))
    
    lixc2881 = float((spw.lixc288[spw.date == q1start]))
    lixc2880 = float((spw.lixc288[spw.date == q0start]))
    lixc288m1 = float((spd.lixc288[spd.date == q1start]))
    lixc288m0 = float((spd.lixc288[spd.date == m0start]))
    q0lixc288.append(float(lixc2881))
    q0lixc288v.append(float(lixc2881 - lixc2880))
    m0lixc288.append(float(lixc288m1))
    m0lixc288v.append(float(lixc288m1 - lixc288m0))
    
    lixd1 = float((spw.lixd[spw.date == q1start]))
    lixd0 = float((spw.lixd[spw.date == q0start]))
    lixdm1 = float((spd.lixd[spd.date == q1start]))
    lixdm0 = float((spd.lixd[spd.date == m0start]))
    q0lixd.append(float(lixd1))
    q0lixdv.append(float(lixd1 - lixd0))
    m0lixd.append(float(lixdm1))
    m0lixdv.append(float(lixdm1 - lixdm0))
    
    lixm1 = float((spw.lixm[spw.date == q1start]))
    lixm0 = float((spw.lixm[spw.date == q0start]))
    lixmm1 = float((spd.lixm[spd.date == q1start]))
    lixmm0 = float((spd.lixm[spd.date == m0start]))
    q0lixm.append(float(lixm1))
    q0lixmv.append(float(lixm1 - lixm0))
    m0lixm.append(float(lixmm1))
    m0lixmv.append(float(lixmm1 - lixmm0))
    
    q1indstart = spw.nrow[spw.date == q1start]
    
    m0cstart = float(spw.c[q1indstart + 4])
    m0cend = q0cend  
    m0cv = ((m0cend/m0cstart)-1)*100
    m0c.append(float(m0cv))
    
    m0clist = spd.c[(spd.date >= m0start) & (spd.date<= q1start)]
    m0clist = m0clist.reset_index(drop=True)
    m0cvars = []
    for i in range(len(m0clist)):
        if i < len(m0clist)-1:
            m0cvars.append(float(((m0clist[i]/m0clist[i+1])-1)*100))
    m0cstd.append(float(np.std(m0cvars)))
    
    m0hlist = spd.h[(spd.date >= m0start) & (spd.date<= q1start)]
    m0hlist = m0hlist.reset_index(drop=True)
    m0hvars = []
    for i in range(len(m0hlist)):
        m0hvars.append(float(((m0hlist[i]/m0clist[i])-1)*100))
    m0high.append(float(np.mean(m0hvars)))
    
    m0llist = spd.l[(spd.date >= m0start) & (spd.date<= q1start)]
    m0llist = m0llist.reset_index(drop=True)
    m0lvars = []
    for i in range(len(m0llist)):
        m0lvars.append(float(((m0llist[i]/m0clist[i])-1)*100))
    m0low.append(float(np.mean(m0lvars)))
    
    m0svars = []
    for i in range(len(m0llist)):
        m0svars.append(float(((m0hlist[i]/m0llist[i])-1)*100))
    m0spanmean.append(float(np.mean(m0svars)))
    m0spanstd.append(float(np.std(m0svars)))

In [257]:
print len(q1c), len(q1cstd), len(q1high), len(q1low), len(q0c), len(q0cstd), len(q0high), len(q0low), len(q0spanmean)

140 140 140 140 140 140 140 140 140


In [258]:
print len(q0spanstd), len(q0lixt), len(q0lixc72), len(q0lixc288), len(q0lixd), len(q0lixm), len(q0lixtv), len(q0lixc72v), len(q0lixc288v)

140 140 140 140 140 140 140 140 140


In [259]:
print len(q0lixdv), len(q0lixmv), len(m0c), len(m0cstd), len(m0high), len(m0low), len(m0spanmean), len(m0spanstd) 

140 140 140 140 140 140 140 140


In [260]:
print len(m0lixt), len(m0lixc72), len(m0lixc288), len(m0lixd), len(m0lixm), len(m0lixtv), len(m0lixc72v), len(m0lixc288v), len(m0lixdv), len(m0lixmv)

140 140 140 140 140 140 140 140 140 140


In [261]:
data = pd.DataFrame(index = exp, columns = ['q1c','q1cstd','q1high','q1low','q0c',
                                            'q0cstd','q0high','q0low','q0spanmean','q0spanstd','q0lixt','q0lixc72',
                                            'q0lixc288','q0lixd','q0lixm','q0lixtv','q0lixc72v','q0lixc288v',
                                            'q0lixdv','q0lixmv','m0c','m0cstd','m0high','m0low','m0spanmean',
                                            'm0spanstd','m0lixt','m0lixc72','m0lixc288','m0lixd','m0lixm','m0lixtv',
                                            'm0lixc72v','m0lixc288v','m0lixdv','m0lixmv'])

In [262]:
data.q1c = q1c
data.q1cstd = q1cstd
data.q1high = q1high
data.q1low = q1low
data.q0c = q0c
data.q0cstd = q0cstd
data.q0high = q0high
data.q0low = q0low
data.q0spanmean = q0spanmean
data.q0spanstd = q0spanstd
data.q0lixt = q0lixt
data.q0lixc72 = q0lixc72
data.q0lixc288 = q0lixc288
data.q0lixd = q0lixd
data.q0lixm = q0lixm
data.q0lixtv = q0lixtv
data.q0lixc72v = q0lixc72v
data.q0lixc288v = q0lixc288v
data.q0lixdv = q0lixdv
data.q0lixmv = q0lixmv
data.m0c = m0c
data.m0cstd = m0cstd
data.m0high = m0high
data.m0low = m0low
data.m0spanmean = m0spanmean
data.m0spanstd = m0spanstd
data.m0lixt = m0lixt
data.m0lixc72 = m0lixc72
data.m0lix288 = m0lixc288
data.m0lixd = m0lixd
data.m0lixm = m0lixm
data.m0lixtv = m0lixtv
data.m0lixc72v = m0lixc72v
data.m0lixc288v = m0lixc288v
data.m0lixdv = m0lixdv
data.m0lixmv = m0lixmv

In [263]:
len(data)

140

In [264]:
# to move weekly date from mondays to next fridays:
#d + DateOffset(weekday = 4)

In [265]:
data.head(5)

,q1c,q1cstd,q1high,q1low,q0c,q0cstd,q0high,q0low,q0spanmean,q0spanstd,...,m0lixt,m0lixc72,m0lixc288,m0lixd,m0lixm,m0lixtv,m0lixc72v,m0lixc288v,m0lixdv,m0lixmv
2015-03-20,1.808621,0.851452,2.257262,-3.765961,2.996910,0.920216,0.512111,-0.576190,1.096336,0.607063,...,7.285306,0,NaN,-5.667367,1,-0.514500,0,0,0.659982,0
2014-12-19,2.996910,0.920216,3.231700,-7.357244,2.421455,0.529902,0.296047,-0.383053,0.681956,0.312472,...,6.239702,0,NaN,-5.549377,1,-0.927099,0,0,0.082600,0
2014-09-19,2.421455,0.529902,2.470362,-2.715414,5.162012,0.599897,0.325310,-0.453131,0.782220,0.379203,...,8.935134,0,NaN,-7.054315,1,2.126565,0,0,-1.652266,0
2014-06-20,5.162012,0.599897,5.162012,-2.723254,2.650802,0.736476,0.401707,-0.473066,0.879293,0.442999,...,7.601946,0,NaN,-6.067856,1,-0.352682,0,0,-0.137991,0
2014-03-21,2.650802,0.736476,3.284355,-4.203327,6.340095,0.660607,0.321345,-0.459026,0.785119,0.386318,...,9.628810,0,NaN,-7.905416,1,-0.422328,0,0,0.764581,0


In [266]:
data.tail(5)

,q1c,q1cstd,q1high,q1low,q0c,q0cstd,q0high,q0low,q0spanmean,q0spanstd,...,m0lixt,m0lixc72,m0lixc288,m0lixd,m0lixm,m0lixtv,m0lixc72v,m0lixc288v,m0lixdv,m0lixmv
1981-03-20,0.284218,0.876694,3.305909,-5.325355,3.442940,1.162911,1.169882,-1.160924,2.358020,0.389244,...,11.163809,0,NaN,-8.391371,1,-3.216641,0.000000,0.000000,5.115661,0
1980-12-19,3.442940,1.162911,8.719536,-4.417795,13.317552,0.848507,0.885664,-1.030819,1.936855,0.258549,...,12.983554,0,NaN,-11.022429,1,-0.725846,0.000000,0.000000,-0.414660,0
1980-09-19,13.317552,0.848507,13.317552,0.000000,11.484703,1.030380,1.051530,-1.165158,2.245430,0.634732,...,5.873999,0,NaN,-5.728638,1,2.612935,3.663191,-1.245592,-1.929930,0
1980-06-20,11.484703,1.030380,13.635031,-3.997654,-4.907519,1.035450,1.184622,-1.101232,2.312078,0.505584,...,-3.365493,0,NaN,4.825966,-1,-11.566640,-0.099965,2.432235,11.860214,-2
1980-03-21,-4.907519,1.035450,10.084580,-4.953992,-2.607043,0.860565,0.921323,-0.903817,1.843019,0.431457,...,3.839171,0,NaN,-3.499358,1,2.506596,0.000000,-0.689170,-1.976643,1


In [267]:
data[100:120]

,q1c,q1cstd,q1high,q1low,q0c,q0cstd,q0high,q0low,q0spanmean,q0spanstd,...,m0lixt,m0lixc72,m0lixc288,m0lixd,m0lixm,m0lixtv,m0lixc72v,m0lixc288v,m0lixdv,m0lixmv
1989-12-15,1.472211,1.048932,4.271718,-3.608071,7.378245,0.745028,0.386147,-0.582260,0.974962,0.432747,...,10.464524,0.000000,NaN,-8.707165,1,-3.194634,0.000000,0.000000,2.246996,0
1989-09-15,7.378245,0.745028,10.076241,-1.048701,9.791933,0.657776,0.352568,-0.509025,0.866914,0.443533,...,9.491387,0.000000,NaN,-8.701740,1,-3.081511,0.000000,0.000000,1.898247,0
1989-06-16,9.791933,0.657776,11.705221,-1.267549,5.935788,0.736357,0.412699,-0.484707,0.902569,0.465407,...,5.246116,0.000000,NaN,-4.961076,1,-3.655708,0.000000,0.000000,2.404960,0
1989-03-17,5.935788,0.736357,8.447642,-0.354703,2.083880,0.752470,0.402490,-0.529930,0.938493,0.473442,...,3.308829,0.000000,NaN,-3.333878,1,6.793257,0.000000,0.644769,-3.403371,1
1988-12-16,2.083880,0.752470,4.806950,-2.523547,-0.011083,0.904268,0.531986,-0.613971,1.154624,0.509275,...,0.891898,0.000000,NaN,-1.556551,1,3.614587,-2.161096,-2.278608,-4.172472,2
1988-09-16,-0.011083,0.904268,1.895231,-5.061320,-0.162293,1.156028,0.629752,-0.657190,1.297418,0.745367,...,-0.282154,0.000000,NaN,-0.898851,0,6.761184,0.000000,-3.240312,-7.454543,1
1988-06-17,-0.162293,1.156028,1.228242,-7.483771,8.813614,1.432713,0.669468,-0.967004,1.655280,1.071096,...,3.376052,0.000000,NaN,0.636119,0,2.946304,0.000000,-2.411686,-3.265101,0
1988-03-18,8.813614,1.432713,8.853749,-2.620806,-20.866414,3.626313,1.737219,-1.424670,3.229012,3.605400,...,-8.614292,0.000000,NaN,12.081775,-1,3.665466,0.000000,-3.333060,-6.629076,0
1987-12-18,-20.866414,3.626313,4.198692,-28.882676,2.570280,0.825903,0.516499,-0.602097,1.126106,0.508756,...,9.563471,1.181667,NaN,-6.483740,1,-8.668820,1.181667,1.181667,7.684689,0
1987-09-18,2.570280,0.825903,9.707786,-1.312832,2.951333,1.152783,0.720856,-0.802392,1.538018,0.787520,...,13.803968,0.000000,NaN,-11.034288,1,9.003631,0.000000,-0.691983,-6.297246,1


In [268]:
data.q0lixt[:5]

2015-03-20    37.516520
2014-12-19    40.196827
2014-09-19    40.484208
2014-06-20    35.310544
2014-03-21    27.761252
Name: q0lixt, dtype: float64

In [269]:
data.to_csv('data.csv')